# 일급 함수
- 파이썬의 **함수**는 일급 객체이다
    - 일급 객체란?
        - 런타임에 생성할 수 있다
        - 데이터 구초제의 변수나 요소에 할당할 수 있다
        - 함수 인수로 전달할 수 있다
        - 함수 결과로 반환할 수 있다
        
> 정수, 딕셔너리, 문자열 들도 일급 객체이다.

## 5.1 함수를 객체처럼 다루기
- 일급 함수가 있으면 함수형 스타일로 프로그래밍 할 수 있다


In [10]:
def factorial(n: int) -> int:
    """return n!"""
    
    return 1 if n < 2 else n * factorial(n - 1)

print(factorial(42))
print(factorial.__doc__)
# 정의한 factorial 함수는 function 클래스의 인스턴스이다!
print(type(factorial))

1405006117752879898543142606244511569936384000000000
return n!
<class 'function'>


In [19]:
fact = factorial
print(fact)
print(map(factorial, range(11)))
print(list(map(fact, range(11))))

<function factorial at 0x7f6cfa7d6b90>
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


## 5.2  고위 함수
- 함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 말한다 (high-order-function)

In [21]:
def reverse(word):
    return word[::-1]

print(sorted(['apple', 'raspberry', 'banana', 'fig', 'strawberry', 'cherry'], key=reverse))

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


In [24]:
# map, filter 고위 함수들은 리스트 컴프리헨션, 제너레이터 컴프리헨션으로 대체할 수 있어 중요성이 떨어졌다
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2])

[1, 6, 120]
[1, 6, 120]


## 5.3 익명 함수 (lambda)
- 람다 함수의 본체는 순수한 표현식으로만 구성되어야 한다
    - while, try 등의 문장 사용 불가능
- 이러한 구문 제한 때문에 고위 함수의 인수로 사용되는 경우를 제외하면 거의 사용되지 않는다

### 5.4 콜러블 객체
- 호출 연산자 ()는 사용자 정의 함수 이외의 다른 객체에도 적용할 수 있다
- 호출할 수 있는 객체인지 알아보려면 `callable()` 내장 함수를 이용하면 된다

In [31]:
print(abs, str, 13)
print([callable(obj) for obj in [abs, str, 13]])

<built-in function abs> <class 'str'> 13
[True, True, False]


## 5.5 사용자 정의 콜러블
- 모든 파이썬 객체가 함수처럼 동작하게 만들 수 있다
- `__call__()` 인스턴스 메서드를 구현하면 된다

In [38]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()
    
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

2
1
True


In [42]:
# 장고 admin 이용 시 설정하는 short_description 속성이 함수의 __dict__ 속성을 이용해 객체에 할당된 사용자 속성을 보관하는 것이다!
print(factorial.__dict__)
factorial.short_description = 'Custom Factorial'
print(factorial.__dict__)

{'short_description': 'Custom Factorial'}
{'short_description': 'Custom Factorial'}


## 5.7 위치 매개변수에서 키워드 전용 매개변수까지
- \* 와 \** 기호로 각각 위치, 키워드 매개변수로 활용할 수 있다

In [54]:
# HTML 을 생성하는 tag함수

def tag(name, *content, cls=None, **attrs):
    """하나 이상의 HTML 태크를 생성한다"""
    
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    
    if content:
        return '\n'.join(f'<{name}{attr_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name}{attr_str} />'

In [66]:
print(tag('br'), '\n')
print(tag('p', 'hello'), '\n')
print(tag('p', 'hello', 'world'), '\n')
print(tag('p', 'hello', id=33), '\n')
print(tag('p', 'hello', 'world', id=33, cls='sidebar'), '\n')

my_tags = {
    'name': 'img',
    'title': 'Sunset Boulevard',
    'src': 'sunset.jpg',
    'cls': 'framed'
}
print(tag(**my_tags), '\n')

<br /> 

<p>hello</p> 

<p>hello</p>
<p>world</p> 

<p id="33">hello</p> 

<p class="sidebar" id="33">hello</p>
<p class="sidebar" id="33">world</p> 

<img class="framed" src="sunset.jpg" title="Sunset Boulevard" /> 



In [67]:
def clip(text, max_len=80):
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다"""
    
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()


In [69]:
clip('adadad  adada  dadadadad   dadadadadadada  adadad dada a  adada ad ad', max_len=30)

'adadad  adada  dadadadad'

## 5.10 함수형 프로그래밍을 위한 패키지

### 5.10.1 operator 모듈

- 수십개의 연산자에 대응하는 함수를 제공한다

In [70]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n + 1))

print(fact(5))

120


In [81]:
# 시퀀스에서 항목을 가져오는 itemgetter() (람다 대체 가능) 과 객체의 속성을 읽는 attrgetter() 함수를 제공
from operator import itemgetter, attrgetter

data = [(1,2, 4), (1,3,1), (1,5, 7), (5,3,1), (4,1,1), (10,13,1)]

# key=lambda x:x[1] 과 동일하다!
print(sorted(data, key=itemgetter(1)))

# n[1::-1] 과 동일
slices = itemgetter(1, 0)
for n in data:
    print(slices(n))
    print(n[1::-1])
    print()

[(4, 1, 1), (1, 2, 4), (1, 3, 1), (5, 3, 1), (1, 5, 7), (10, 13, 1)]
(2, 1)
(2, 1)

(3, 1)
(3, 1)

(5, 1)
(5, 1)

(3, 5)
(3, 5)

(1, 4)
(1, 4)

(13, 10)
(13, 10)



### 5.10.2 functools.partial 함수로 인수 고정하기

In [84]:
from functools import partial

# tag 함수에 img, cls='pic-frame' 인수가 들어간 상태로 고정된다.
picture = partial(tag, 'img', cls='pic-frame')

print(picture(src='wumpus.jpeg'))

<img class="pic-frame" src="wumpus.jpeg" />
